In [18]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Reshape, Flatten, RepeatVector, LSTM
from time import gmtime, strftime
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Data loading and processing

In [3]:
# onehot encode
onehot_label = ["0","1","2","3","4","5","6","7","8","9","+","-"," "]
encoder = LabelEncoder()
encoder.fit(onehot_label)

LabelEncoder()

In [4]:
# onehot encoder
def one_hotencoder(dataset):
    dataset_onehot = []
    for data in dataset:
        data_onehot = encoder.transform(list(data))
        data_onehot = np_utils.to_categorical(data_onehot, num_classes=len(onehot_label))
        dataset_onehot.append(data_onehot)
    return dataset_onehot

In [5]:
# onehot decoder
def decoder(list):
    label_index = np.argmax(list, axis=1)
    label = encoder.inverse_transform(label_index)
    #return label
    return "".join(i for i in label)

In [6]:
x_train_path = "data/trainData.npy"
y_train_path = "data/trainTarget.npy"

In [7]:
def loadnpy(path):
    data = np.load(path)
    return data

In [8]:
x_train = loadnpy(x_train_path)
y_train = loadnpy(y_train_path)

# (7, 13)
#print(x_train[0].shape)
# (4, 13)
#print(y_train[0].shape)

# Build Model

In [15]:
def bulidModel(x_shape):
    model = Sequential()
    model.add(Dense(units=256, input_shape=(7, 13), activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=64,activation='relu'))
    model.add(Dense(units=32,activation='relu'))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dense(units=52,activation='softmax'))
    model.add(Reshape((4, 13)))
    model.summary()
    
    return model


In [27]:
def bulidLSTMModel(x_shape):
    model = Sequential()
    model.add(LSTM(units=256, input_shape=(7, 12), activation='tanh'))
    model.add(RepeatVector(4))
    model.add(LSTM(units=128, activation='tanh', return_sequences=True))
    model.add(Dense(units=128, activation='sigmoid'))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=12, activation='softmax'))
    model.summary()
    
    return model

In [50]:
print("Build Model")

model = bulidModel(x_train.shape)
model.compile(loss='categorical_crossentropy', optimizer='adam')#adam

print("Model Training")
model_history = model.fit(x_train , y_train , batch_size=128 , epochs=5 , verbose=1, validation_split=0.2, shuffle=True)

time = strftime("%Y%m%d_%H_%M", gmtime())

model.save('model_'+time+'.h5')

Build Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 256)               276480    
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 4, 256)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 4, 39)             10023     
_________________________________________________________________
dense_18 (Dense)             (None, 4, 13)             520       
Total params: 287,023
Trainable params: 287,023
Non-trainable params: 0
_________________________________________________________________
Model Training
Train on 1600000 samples, validate on 400000 samples
Epoch 1/5
 658688/1600000 [===========>..................] - ETA: 2:33 - loss: 2.0678

KeyboardInterrupt: 

# Testing

In [36]:
x_test_path = "data/x_test.npy"
y_test_path = "data/y_test.npy"
#model_path = "model_20190429_14_44.h5"

In [37]:
x_test = loadnpy(x_test_path)
y_test = loadnpy(y_test_path)
#model = load_model(model_path)

In [42]:
train_score = model.evaluate(x_train, y_train)

print("training score :", train_score)


2000000/2000000 [==============================] - 446s 223us/step
training score : 1.7937548957824707
testing score : 1.7937548957824707


In [46]:
test_score = model.evaluate(x_test, y_test)
print("testing score :", test_score)

1000/1000 [==============================] - 0s 250us/step
testing score : 1.3856794576644897


In [47]:
output_mse = model.predict(x_test)
output = model.predict_classes(x_test)
#print(output_mse[0])

In [48]:
mse = ((y_test - output_mse)**2).mean(axis=None)
print("MSE : ", mse) 
#mse : 0.048510086 for model model_20190429_14_44.h5

MSE :  0.05606687


In [49]:
count = 0
for i in range(len(y_test)):
    q = decoder(x_test[i])
    guess = decoder(output_mse[i])
    correct = decoder(y_test[i])
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    #print("\n", correct, guess)
    if correct == guess:
        print('☑', end=' ')
        count += 1
    else:
        print('☒', end=' ')
    print(guess)

print("Precision : ", count/1000)

Q 0+0     T 0    ☒     
Q 0-0     T 0    ☒     
Q 0+1     T 1    ☒     
Q 0-1     T -1   ☒     
Q 0+2     T 2    ☒     
Q 0-2     T -2   ☒     
Q 0+3     T 3    ☒     
Q 0-3     T -3   ☒     
Q 0+4     T 4    ☒     
Q 0-4     T -4   ☒     
Q 0+5     T 5    ☒     
Q 0-5     T -5   ☒     
Q 0+6     T 6    ☒     
Q 0-6     T -6   ☒     
Q 0+7     T 7    ☒     
Q 0-7     T -7   ☒     
Q 0+8     T 8    ☒     
Q 0-8     T -8   ☒     
Q 0+9     T 9    ☒     
Q 0-9     T -9   ☒     
Q 0+10    T 10   ☒     
Q 0-10    T -10  ☒ 1111
Q 0+11    T 11   ☒     
Q 0-11    T -11  ☒ 1111
Q 0+12    T 12   ☒     
Q 0-12    T -12  ☒ 1111
Q 0+13    T 13   ☒     
Q 0-13    T -13  ☒ ----
Q 0+14    T 14   ☒     
Q 0-14    T -14  ☒ ----
Q 0+15    T 15   ☒     
Q 0-15    T -15  ☒ ----
Q 0+16    T 16   ☒     
Q 0-16    T -16  ☒ ----
Q 0+17    T 17   ☒     
Q 0-17    T -17  ☒ 1111
Q 0+18    T 18   ☒     
Q 0-18    T -18  ☒ 1111
Q 0+19    T 19   ☒     
Q 0-19    T -19  ☒ 1111
Q 0+20    T 20   ☒     
Q 0-20    T -20 

Q 0-256   T -256 ☒ 6666
Q 0+257   T 257  ☒ 7777
Q 0-257   T -257 ☒ ----
Q 0+258   T 258  ☒ 8888
Q 0-258   T -258 ☒ 2222
Q 0+259   T 259  ☒ 2222
Q 0-259   T -259 ☒ 5555
Q 0+260   T 260  ☒ 0000
Q 0-260   T -260 ☒ 2222
Q 0+261   T 261  ☒ 1111
Q 0-261   T -261 ☒ 6666
Q 0+262   T 262  ☒ 2222
Q 0-262   T -262 ☒ 2222
Q 0+263   T 263  ☒ 6666
Q 0-263   T -263 ☒ 3333
Q 0+264   T 264  ☒     
Q 0-264   T -264 ☒ 6666
Q 0+265   T 265  ☒     
Q 0-265   T -265 ☒ 2222
Q 0+266   T 266  ☒ 2222
Q 0-266   T -266 ☒ 6666
Q 0+267   T 267  ☒ 7777
Q 0-267   T -267 ☒ 6666
Q 0+268   T 268  ☒ 8888
Q 0-268   T -268 ☒ 8888
Q 0+269   T 269  ☒ 2222
Q 0-269   T -269 ☒ 9999
Q 0+270   T 270  ☒ 7777
Q 0-270   T -270 ☒ 2222
Q 0+271   T 271  ☒ 2222
Q 0-271   T -271 ☒ 2222
Q 0+272   T 272  ☒ 2222
Q 0-272   T -272 ☒ 2222
Q 0+273   T 273  ☒ 3333
Q 0-273   T -273 ☒ 7777
Q 0+274   T 274  ☒ 4444
Q 0-274   T -274 ☒ 4444
Q 0+275   T 275  ☒     
Q 0-275   T -275 ☒ 7777
Q 0+276   T 276  ☒ 2222
Q 0-276   T -276 ☒ 2222
Q 0+277   T 277 